# Imports

In [1]:
import os
import gc
import re
import time
import random
import logging
import warnings
import tempfile
import subprocess
import requests
import json

import pandas as pd
import polars as pl

import torch
import kaggle_evaluation.aimo_2_inference_server

from transformers import set_seed, AutoTokenizer

from huggingface_hub import notebook_login, login

from vllm import LLM, SamplingParams

from collections import Counter, defaultdict

from multiprocessing import Process, Queue, Event
import multiprocessing

from tqdm import tqdm

In [5]:
!ls /kaggle/input/qwen2.5/transformers/72b-instruct-awq/1

config.json			  model-00007-of-00011.safetensors
generation_config.json		  model-00008-of-00011.safetensors
LICENSE				  model-00009-of-00011.safetensors
merges.txt			  model-00010-of-00011.safetensors
model-00001-of-00011.safetensors  model-00011-of-00011.safetensors
model-00002-of-00011.safetensors  model.safetensors.index.json
model-00003-of-00011.safetensors  README.md
model-00004-of-00011.safetensors  tokenizer_config.json
model-00005-of-00011.safetensors  tokenizer.json
model-00006-of-00011.safetensors  vocab.json


In [4]:
!ls /kaggle/input/xct-ea-mt/xct

predictions  references


# Setup

In [ ]:
from huggingface_hub import notebook_login, login

login(token='*******')

In [2]:
# fixed seed to get similar score
set_seed(42)

pd.set_option('display.max_colwidth', None)
cutoff_time = time.time() + (4 * 60 + 45) * 60

warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["VLLM_ALLOW_LONG_MAX_MODEL_LEN"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

model_path = "/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1"
data_path = "/kaggle/input/xct-ea-mt/xct"
output_path = "/kaggle/working/xct/"

languages = {
    "ar": "Arabic",
    "zh": "Chinese (Traditional)",
    "fr": "French",
    "de": "German",
    "it": "Italian",
    "ja": "Japanese",
    "ko": "Korean",
    "es": "Spanish",
    "th": "Thai",
    "tr": "Turkish",
    "en": "English"
}

In [3]:
sampling_params = SamplingParams(
    temperature=0.8,              # Controls randomness in generation: higher values (e.g., 1.0) produce more diverse output.
    top_p = 0.85,
    repetition_penalty=1.1,
    min_p=0.01,                   # Minimum cumulative probability for nucleus sampling, filtering out unlikely tokens.
    skip_special_tokens=True,     
    max_tokens=32768,             # Sets a very high limit for token generation to handle longer outputs.
    # stop=["```output"],          # List of strings that stop the generation when they are generated.
)

# Load Models

In [4]:
llm_model_path = model_path

llm = LLM(
    model=llm_model_path,
    # dtype="half",                -> Changed this
    # max_num_seqs=128,            -> Changed this
    max_model_len=32768,         # 131072 for qwen 2.5 math instruct,  
    trust_remote_code=True,     
    tensor_parallel_size=4,       # number of GPUs to use for distributed execution
    gpu_memory_utilization=0.96, 
)

INFO 01-06 13:31:04 awq_marlin.py:97] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 01-06 13:31:04 config.py:905] Defaulting to use mp for distributed inference
INFO 01-06 13:31:04 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_conf

Loading safetensors checkpoint shards:   0% Completed | 0/11 [00:00<?, ?it/s]


(VllmWorkerProcess pid=544) INFO 01-06 13:35:49 model_runner.py:1067] Loading model weights took 9.6969 GB
(VllmWorkerProcess pid=545) INFO 01-06 13:35:49 model_runner.py:1067] Loading model weights took 9.6969 GB
INFO 01-06 13:35:49 model_runner.py:1067] Loading model weights took 9.6969 GB
(VllmWorkerProcess pid=543) INFO 01-06 13:35:49 model_runner.py:1067] Loading model weights took 9.6969 GB
INFO 01-06 13:36:41 distributed_gpu_executor.py:57] # GPU blocks: 5998, # CPU blocks: 3276
INFO 01-06 13:36:41 distributed_gpu_executor.py:61] Maximum concurrency for 32768 tokens per request: 2.93x
INFO 01-06 13:36:44 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-06 13:36:44 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_me

In [5]:
tokenizer = llm.get_tokenizer()

In [13]:
result = llm.generate("Write the following sentence with named entities separated by the tags <NER> and </NER>, do not do anything else: Does John Doe uses his shotgun to shoot monsters in From?", sampling_params=sampling_params)

answer = result[0].outputs[0].text

print("Answer:", answer)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.86s/it, est. speed input: 19.92 toks/s, output: 19.39 toks/s]

Answer:  
Does <NER>John Doe</NER> uses his shotgun to shoot monsters in <NER>From</NER>?  (assuming From is a location or game name)


# Utlities

In [6]:
system_prompt = """You are an advanced language model skilled at identifying and isolating named entities in a sentence."""
user_prompt = """Given a sentence, perform the following tasks:

Identify the named entities in the sentence.
Encapsulate each named entity between <NER_{number}> and </NER_{number}> tags, where number indicates the order of the entity found.
Encapsulate the entire sentence, with the named entity tags included, between <SENTENCE> and </SENTENCE> tags.

Example:
Input: Barack Obama was born in Hawaii.
Output: <SENTENCE> <NER_1>Barack Obama</NER_1> was born in <NER_2>Hawaii</NER_2>. </SENTENCE>

Task:
Input: %(input_sentence)s
Output:"""

translate_system_prompt = """You are a highly skilled language model capable of translating text between languages with high accuracy.
Translate sentences into the specified target language while preserving their meaning and context.
Do not translate the parts of the sentence enclosed between <NER> and </NER> tags."""
translate_user_prompt = """Translate the following sentence into %(target_language)s:\n"
Sentence: %(sentence)s
Translation:"""

validate_system_prompt = """You are an expert in evaluating the fluency and naturalness of sentences in a specific language. 
Your task is to determine whether a provided sentence sounds natural and fluent in the target language. 
If the sentence is already fluent and natural, return it as is. 
Do not provide explanations or reasoning.
If minor adjustments are needed for fluency, provide the refined sentence in the target language.
The target language is %(language)s.
"""
validate_user_prompt = """The following sentence is in %(language)s. 
Please evaluate whether it sounds natural and fluent in the target language. 

Translated Sentence: %(translated)s Final fluent sentence: """

In [4]:
print(user_prompt % {"input_sentence": "This is my sentence"})

Given a sentence, perform the following tasks:

Identify the named entities in the sentence.
Encapsulate each named entity between <NER> and </NER> tags.
Encapsulate the entire sentence, with the named entity tags included, between <SENTENCE> and </SENTENCE> tags.

Example:
Input: Barack Obama was born in Hawaii.
Output: <SENTENCE> <NER>Barack Obama</NER> was born in <NER>Hawaii</NER>. </SENTENCE>

Task:
Input: This is my sentence
Output:


In [7]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f]

def save_jsonl(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def get_language_pair(file_name):
    target_language_code = file_name.split('.')[0]
    source_language_code = "en" 
    return source_language_code, target_language_code.split('_')[0]

# Text Generation Functions

In [8]:
# generate prompts in batch
def batch_message_generate(llm, tokenizer, list_of_messages) -> list[list[dict]]:
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )

    # print('Model output:')
    # print(f'No. prompts: {len(list_of_texts)} | no. outputs: {len(request_output)} | no. text: {len([response.text for output in request_output for response in output.outputs])}')
    # print(*[response.text for output in request_output for response in output.outputs], sep="\n--------------------------------------------------\n")
    
    # print('Messages: ')
    for messages, single_request_output in zip(list_of_messages, request_output):
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})
        # print(messages)

    # print(*list_of_messages, sep="\n-----------------------------------------------\n")
    return list_of_messages

In [9]:
def extract_sentence(text):
    # Regular expression to match text between <SENTENCE> and </SENTENCE>
    match = re.search(r"<SENTENCE>(.*?)</SENTENCE>", text, re.DOTALL)
    if match:
        return match.group(1).strip()  # Extract the sentence and remove extra whitespace
    return None  # Return None if no match is found

In [10]:
def extract_named_entities(text):
    # Regular expression to match content inside <NER_{number}>...</NER_{number}>
    matches = re.findall(r'<NER_\d+>(.*?)</NER_\d+>', text)
    return [match.strip() for match in matches]  # Remove extra whitespace

In [11]:
def get_entity_translation(entity_name, target_language):
    try:
        # 1. Search for the entity in Wikidata
        search_url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&format=json&language=en&type=item&search={entity_name}"
        search_response = requests.get(search_url)
        search_response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        search_data = search_response.json()

        if not search_data.get("search"):
            return None  # Entity not found

        first_result = search_data["search"][0]
        entity_qid = first_result["id"]

        # 2. Get the entity data, including labels
        entity_url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={entity_qid}&props=labels&languages={target_language}"
        entity_response = requests.get(entity_url)
        entity_response.raise_for_status()
        entity_data = entity_response.json()

        entity = entity_data["entities"].get(entity_qid)
        if not entity:
            return None
        labels = entity.get("labels")
        # print(f'Labels: {labels}')

        if labels and labels.get(target_language):
            translated_label = labels[target_language]["value"]
            return translated_label
        else:
            return None # Translation not found

    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Wikidata API: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON response from Wikidata: {e}")
        return None
    except IndexError:
        print("Unexpected response format from Wikidata")
        return None

In [12]:
def replace_named_entities(sentence, entities):
    # Function to replace entities in the new sentence based on the tags
    def replacer(match):
        # Extract the number from the tag
        number = int(match.group(1)) - 1  # Convert to 0-based index
        if 0 <= number < len(entities):
            return entities[number]
        return match.group(0)  # Return original tag if number is out of range

    # Regular expression to match tags like <NER_{number}>...</NER_{number}>
    pattern = r'<NER_(\d+)>.*?</NER_\d+>'
    # Substitute the tags with the corresponding entity
    result = re.sub(pattern, replacer, sentence)
    return result

In [13]:
def solve_step(system_prompt, user_prompt):
    prompts = [
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    ]
    # print('Step 1 prompts:')
    # print(*step1_prompts, sep='\n---------------------------------------------------------\n')

    # print(f'Prompt: {prompts}')
    messages = batch_message_generate(llm, tokenizer, prompts)
    # print('Step 1 message results:')
    # print(*messages, sep='\n---------------------------------------------------------\n')
    return messages

In [14]:
def predict_for_question(question, target_language, target_language_code):
    # print(f'target language: {target_language}')
    step1_answer = solve_step(system_prompt, user_prompt % {"input_sentence": question})
    step1_answer = step1_answer[0][2]['content']
    # print(f'Step1 answer: {step1_answer} \n-----------------------------------')
    step1_answer = extract_sentence(step1_answer)
    # print(f'Step1 answer: {step1_answer} \n-----------------------------------')

    step2_answer = solve_step(translate_system_prompt, translate_user_prompt % {"target_language": target_language, "sentence": step1_answer})
    step2_answer = step2_answer[0][2]['content']
    # print(f'Step2 answer: {step2_answer} \n-----------------------------------')

    step3_answer = extract_named_entities(step1_answer)
    # print(f'Step3 answer: {step3_answer} \n-----------------------------------')

    step4_answer = [get_entity_translation(answer, target_language_code) for answer in step3_answer]
    # print(f'Step4 answer: {step4_answer} \n-----------------------------------')

    step5_answer = replace_named_entities(step2_answer, step4_answer)
    # print(f'Step5 answer: {step5_answer} \n-----------------------------------')

    step6_answer = solve_step(validate_system_prompt % {"language": target_language}, validate_user_prompt % {"language": target_language, "sentence": question, "translated": step5_answer})
    step6_answer = step6_answer[0][2]['content']
    # print(f'Step6 answer: {step6_answer} \n-----------------------------------')

    return step6_answer

In [15]:
def process_file(file_path, output_dir):
    data = load_jsonl(file_path)
    source_language_code, target_language_code = get_language_pair(os.path.basename(file_path))

    output_data = []

    for item in tqdm(data, desc=f"Processing {file_path}"):
        question = item['source']

        prediction = predict_for_question(question, languages[target_language_code], target_language_code)

        output_data.append({
            "id": item["id"],
            "source_language": languages[source_language_code],
            "target_language": languages[target_language_code],
            "text": question,
            "prediction": prediction
        })

    output_file = os.path.join(output_dir, os.path.basename(file_path))
    save_jsonl(output_data, output_file)

In [16]:
def process_data(reference_dir, output_dir, model_name, file_to_process=None):
    input_dir = os.path.join(reference_dir, "validation")
    output_dir = os.path.join(output_dir, model_name, "validation")

    os.makedirs(output_dir, exist_ok=True)

    if file_to_process: 
        file_path = os.path.join(input_dir, file_to_process)
        if file_to_process.endswith(".jsonl") and os.path.isfile(file_path):
            process_file(file_path, output_dir)
        else:
            print(f"Specified file {file_to_process} not found or invalid.")
    else:  # Process all files
        for file_name in os.listdir(input_dir):
            file_path = os.path.join(input_dir, file_name)
            if file_name.endswith(".jsonl"):
                process_file(file_path, output_dir)

# Playground

In [20]:
# question = "Can you sing the chorus of the folk song Ring a Ring o' Roses?"
# question = "When was the film Little Women released?"
question = "Who is the author of the book The Prize: The Epic Quest for Oil, Money, and Power?"
step1_answer = solve_step(system_prompt, user_prompt % {"input_sentence": question})
print(step1_answer)
print('----------------------------------------')
step1_answer = step1_answer[0][2]['content']
step1_answer = extract_sentence(step1_answer)
print(step1_answer)
print('----------------------------------------')
step2_answer = solve_step(translate_system_prompt, translate_user_prompt % {"target_language": "Japanese", "sentence": step1_answer})
step2_answer = step2_answer[0][2]['content']
print(step2_answer)
print('----------------------------------------')
step3_answer = extract_named_entities(step1_answer)
print(step3_answer)
print('----------------------------------------')
step4_answer = [get_entity_translation(answer, "ja") for answer in step3_answer]
print(step4_answer)
print('----------------------------------------')
step5_answer = replace_entities_in_sentence(step2_answer, step3_answer, step4_answer)
print(step5_answer)
print('----------------------------------------')
target_language = "ja"
step6_answer = solve_step(validate_system_prompt % {"language": target_language}, validate_user_prompt % {"language": target_language, "sentence": question, "translated": step5_answer})
step6_answer = step6_answer[0][2]['content']
print(step6_answer)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.91s/it, est. speed input: 82.89 toks/s, output: 17.84 toks/s]


[[{'role': 'system', 'content': 'You are an advanced language model skilled at identifying and isolating named entities in a sentence.'}, {'role': 'user', 'content': 'Given a sentence, perform the following tasks:\n\nIdentify the named entities in the sentence.\nEncapsulate each named entity between <NER> and </NER> tags.\nEncapsulate the entire sentence, with the named entity tags included, between <SENTENCE> and </SENTENCE> tags.\n\nExample:\nInput: Barack Obama was born in Hawaii.\nOutput: <SENTENCE> <NER>Barack Obama</NER> was born in <NER>Hawaii</NER>. </SENTENCE>\n\nTask:\nInput: Who is the author of the book The Prize: The Epic Quest for Oil, Money, and Power?\nOutput:'}, {'role': 'assistant', 'content': '<SENTENCE> Who is the author of the book <NER>The Prize: The Epic Quest for Oil, Money, and Power</NER>? </SENTENCE>'}]]
----------------------------------------
Who is the author of the book <NER>The Prize: The Epic Quest for Oil, Money, and Power</NER>?
----------------------

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it, est. speed input: 67.15 toks/s, output: 17.95 toks/s]


<NER>The Prize: The Epic Quest for Oil, Money, and Power</NER>という本の著者は誰ですか？
----------------------------------------
['The Prize: The Epic Quest for Oil, Money, and Power']
----------------------------------------
['石油の世紀 - 支配者たちの興亡（上下）']
----------------------------------------
石油の世紀 - 支配者たちの興亡（上下）という本の著者は誰ですか？
----------------------------------------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it, est. speed input: 121.14 toks/s, output: 16.68 toks/s]

『石油、お金、そして権力の壮大な追求』という本の著者は誰ですか？


In [23]:
file_path = '/kaggle/input/xct-ea-mt/xct/references/validation/ja_JP.jsonl'
source_language_code, target_language_code = get_language_pair(os.path.basename(file_path))
print(source_language_code)
print(target_language_code)
print(languages[target_language_code])
prediction = predict_for_question("Who is the author of the book The Prize: The Epic Quest for Oil, Money, and Power?", languages[target_language_code], target_language_code)
print(prediction)

## TODO: sometimes the models also translates the Named Entities, extract a map of NER_1 : entity and replace it in the sentence

en
ja
Japanese
target language: Japanese
Prompt: [[{'role': 'system', 'content': 'You are an advanced language model skilled at identifying and isolating named entities in a sentence.'}, {'role': 'user', 'content': 'Given a sentence, perform the following tasks:\n\nIdentify the named entities in the sentence.\nEncapsulate each named entity between <NER_{number}> and </NER_{number}> tags, where number indicates the order of the entity found.\nEncapsulate the entire sentence, with the named entity tags included, between <SENTENCE> and </SENTENCE> tags.\n\nExample:\nInput: Barack Obama was born in Hawaii.\nOutput: <SENTENCE> <NER_1>Barack Obama</NER_1> was born in <NER_2>Hawaii</NER_2>. </SENTENCE>\n\nTask:\nInput: Who is the author of the book The Prize: The Epic Quest for Oil, Money, and Power?\nOutput:'}]]


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.13s/it, est. speed input: 84.41 toks/s, output: 17.82 toks/s]


Step1 answer: <SENTENCE> Who is the author of the book <NER_1>The Prize: The Epic Quest for Oil, Money, and Power</NER_1>? </SENTENCE> 
-----------------------------------
Step1 answer: Who is the author of the book <NER_1>The Prize: The Epic Quest for Oil, Money, and Power</NER_1>? 
-----------------------------------
Prompt: [[{'role': 'system', 'content': 'You are a highly skilled language model capable of translating text between languages with high accuracy.\nTranslate sentences into the specified target language while preserving their meaning and context.\nDo not translate the parts of the sentence enclosed between <NER> and </NER> tags.'}, {'role': 'user', 'content': 'Translate the following sentence into Japanese:\n"\nSentence: Who is the author of the book <NER_1>The Prize: The Epic Quest for Oil, Money, and Power</NER_1>?\nTranslation:'}]]


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it, est. speed input: 65.33 toks/s, output: 18.04 toks/s]


Step2 answer: <NER_1>The Prize: The Epic Quest for Oil, Money, and Power</NER_1>の著者は誰ですか？ 
-----------------------------------
Step3 answer: ['The Prize: The Epic Quest for Oil, Money, and Power'] 
-----------------------------------
Step4 answer: ['石油の世紀 - 支配者たちの興亡（上下）'] 
-----------------------------------
Step5 answer: 石油の世紀 - 支配者たちの興亡（上下）の著者は誰ですか？ 
-----------------------------------
Prompt: [[{'role': 'system', 'content': 'You are an expert in evaluating the fluency and naturalness of sentences in a specific language. \nYour task is to determine whether a provided sentence sounds natural and fluent in the target language. \nIf the sentence is already fluent and natural, return it as is. \nDo not provide explanations or reasoning.\nIf minor adjustments are needed for fluency, provide the refined sentence in the target language.\nThe target language is Japanese.\n'}, {'role': 'user', 'content': 'The following sentence is in Japanese. \nPlease evaluate whether it sounds natural and f

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.36s/it, est. speed input: 111.50 toks/s, output: 16.87 toks/s]

Step6 answer: 石油の世紀 - 支配者たちの興亡（上下）の著者は誰ですか？ 
-----------------------------------
石油の世紀 - 支配者たちの興亡（上下）の著者は誰ですか？


In [27]:
!ls /kaggle/working/xct/predictions/QWEN/validation

ja_JP.jsonl


In [18]:
from IPython.display import FileLink
FileLink(r'xct/predictions/QWEN/validation/it_IT.jsonl')

/kaggle/working/xct/predictions/QWEN/validation/it_IT.jsonl

In [17]:
process_data(os.path.join(data_path, "references"), os.path.join(output_path, "predictions"), "QWEN", "it_IT.jsonl")

Processing /kaggle/input/xct-ea-mt/xct/references/validation/it_IT.jsonl:   0%|          | 0/730 [00:00<?, ?it/s]
Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it, est. speed input: 94.02 toks/s, output: 15.11 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it, est. speed input: 75.35 toks/s, output: 17.63 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s, est. speed input: 154.34 toks/s, output: 15.43 toks/s]
Processing /kaggle/input/xct-ea-mt/xct/references/validation/it_IT.jsonl:   0%|          | 1/730 [00:04<53:12,  4.38s/it]
Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it, est. speed input: 77.02 toks/s, output: 18.02 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it, est. speed input: 61.84 toks/s, output: 17.92 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s, est. speed input: 171.12 toks/s, output: 14.77 toks/s]
Processing /kaggle/input/xct-ea-mt/xct/refer

# Evalution